<a href="https://colab.research.google.com/github/Elshan777/Text-Summarization-for-Lecture-Notes/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of Text Summarization Models with new evaluation method


In [1]:
!git clone https://github.com/dmmiller612/lecture-summarizer.git

Cloning into 'lecture-summarizer'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 259 (delta 0), reused 3 (delta 0), pack-reused 255
Receiving objects: 100% (259/259), 280.43 KiB | 18.69 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [2]:
!git clone https://github.com/nlp-with-transformers/notebooks.git
%cd notebooks
from install import *
install_requirements(is_chapter6=True)

Cloning into 'notebooks'...
remote: Enumerating objects: 507, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 507 (delta 100), reused 100 (delta 87), pack-reused 378
Receiving objects: 100% (507/507), 30.39 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/notebooks
⏳ Installing base requirements ...
✅ Base requirements installed!
⏳ Installing Git LFS ...
✅ Git LFS installed!


In [19]:
#hide_output
import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from tqdm import tqdm

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Reading the data

In [4]:
with open('/content/lecture-summarizer/data/news.txt') as f:
    contents = f.read()
news = contents

with open('/content/lecture-summarizer/data/health.txt') as f:
    contents = f.read()
health = contents

with open('/content/lecture-summarizer/data/health_today.txt') as f:
    contents = f.read()
health_today = contents

with open('/content/lecture-summarizer/data/health_today_1.txt') as f:
    contents = f.read()
health_today_1 = contents

### Creating Dataframe from all the data

In [5]:
import pandas as pd

df = pd.DataFrame(data = {'name': ['news', 'health', 'health_today', 'health_today_1'],
                    'content': [news, health, health_today, health_today_1] })
df.head()

,name,content
0,news,"The Chrysler Building, the famous art deco New..."
1,health,dr karen bell is the chair of the certificatio...
2,health_today,The most sophisticated form of HIE creates sem...
3,health_today_1,"In 2013, Grahame Greive, the leader of the gl..."


## Define Evaluation method

In [6]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=6d9c15563f9b40a3a1498fda5837487dadb6d1db78b6092d6696ebe7e9c87930
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [8]:
def cosine_similarity(a, b):
    nominator = np.dot(a, b)
    
    a_norm = np.sqrt(np.sum(a**2))
    b_norm = np.sqrt(np.sum(b**2))
    
    denominator = a_norm * b_norm
    
    cosine_similarity = nominator / denominator
    
    return cosine_similarity

In [9]:
def evaluate(model, paragraph, summary):
  # Get sentence embedding for paragraph
  paragraph_embeddings = model.encode(sent_tokenize(paragraph))

  # Get sentence embedding for summary
  summary_embeddings = model.encode(sent_tokenize(summary))

  result = np.array([])
  # loop through summary sentences and find arg max cosine similarity
  for summary in summary_embeddings:
    max = 0
    for paragraph in paragraph_embeddings:
      similarity = cosine_similarity(summary, paragraph) 
      if  similarity > max:
        max = similarity
    result = np.append(result, max)
  # return the mean
  return np.mean(result)

In [60]:
import textwrap

def generate_summary(paragraph):
  arr = textwrap.wrap(paragraph, 5000)
  summary = ''
  for i in arr:
    summary += pipe( i ) [0]["summary_text"].replace(" .<n>", ".\n")
  return summary

## BART

In [10]:
from transformers import pipeline, set_seed

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [55]:
bart_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary(df['content'][i] ) 
  bart_summaries.append( summary )

df['bart'] = bart_summaries
df['bart_score'] = df[['content', 'bart']].apply(lambda x: evaluate(model, x['content'], x['bart']), axis=1)

100%|██████████| 4/4 [03:46<00:00, 56.56s/it]


In [58]:
df.head()

,name,content,bart_score,bart
0,news,"The Chrysler Building, the famous art deco New...",0.965930,"The deal was for $150 million, according to a ..."
1,health,dr karen bell is the chair of the certificatio...,0.481145,dr karen bell is the chair of the certificati...
2,health_today,The most sophisticated form of HIE creates sem...,0.825120,The most sophisticated form of HIE creates sem...
3,health_today_1,"In 2013, Grahame Greive, the leader of the gl...",0.930182,The HL7 task force FHIR does not attempt to ad...


## Pegasus CNN Daily Mail

In [59]:
#hide_output
from transformers import pipeline, set_seed
#hide_output

pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [61]:
pegasus_summaries = []
for i in tqdm(range( len(df) ) ):
  summary = generate_summary(df['content'][i] ) 
  pegasus_summaries.append( summary )

df['pegasus'] = pegasus_summaries
df['pegasus_score'] = df[['content', 'pegasus']].apply(lambda x: evaluate(model, x['content'], x['pegasus']), axis=1)

100%|██████████| 4/4 [08:08<00:00, 122.10s/it]


In [63]:
df

,name,content,bart_score,bart,pegasus,pegasus_score
0,news,"The Chrysler Building, the famous art deco New...",0.965930,"The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,0.810819
1,health,dr karen bell is the chair of the certificatio...,0.481145,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,0.443118
2,health_today,The most sophisticated form of HIE creates sem...,0.825120,The most sophisticated form of HIE creates sem...,The Indiana Health Information Exchange is the...,0.787513
3,health_today_1,"In 2013, Grahame Greive, the leader of the gl...",0.930182,The HL7 task force FHIR does not attempt to ad...,There are many strongly held points of view ab...,1.000000


## Pegasus XSUM

In [ ]:
#hide_output
from transformers import pipeline, set_seed

pipe = pipeline("summarization", model="google/pegasus-xsum")

In [ ]:
df['pegasus_xsum'] = df['content'].apply(lambda x: (pipe(x, truncation=True)[0]["summary_text"].replace(" .<n>", ".\n")) )
df['pegasus_xsum_score'] = df[['content', 'pegasus_xsum']].apply(lambda x: evaluate(model, x['content'], x['pegasus_xsum']), axis=1)
df.head()

"One of the world's tallest office buildings has been sold."

In [ ]:
evaluate(model, sample_text, summaries["pegasus_xsum"])

0.582875669002533

In [ ]:
sample_text

"The Chrysler Building, the famous art deco New York skyscraper, will be sold for a small fraction of its previous sales price.\nThe deal, first reported by The Real Deal, was for $150 million, according to a source familiar with the deal.\nMubadala, an Abu Dhabi investment fund, purchased 90% of the building for $800 million in 2008.\nReal estate firm Tishman Speyer had owned the other 10%.\nThe buyer is RFR Holding, a New York real estate company.\nOfficials with Tishman and RFR did not immediately respond to a request for comments.\nIt's unclear when the deal will close.\nThe building sold fairly quickly after being publicly placed on the market only two months ago.\nThe sale was handled by CBRE Group.\nThe incentive to sell the building at such a huge loss was due to the soaring rent the owners pay to Cooper Union, a New York college, for the land under the building.\nThe rent is rising from $7.75 million last year to $32.5 million this year to $41 million in 2028.\nMeantime, rents

## Blanc Metric

In [62]:
!pip install blanc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from blanc import BlancHelp

blanc_help = BlancHelp()

blanc_help.eval_once(document, summary)

# def blanc_eval():

In [ ]:
df['pegasus_blanc'] = df[['content', 'pegasus']].apply(lambda x: blanc_help.eval_once(x['content'], x['pegasus']), axis=1)

In [ ]:
df['bart_blanc'] = df[['content', 'bart']].apply(lambda x: blanc_help.eval_once(x['content'], x['bart']), axis=1)

In [70]:
df.head()

,name,content,bart_score,bart,pegasus,pegasus_score,pegasus_blanc,bart_blanc
0,news,"The Chrysler Building, the famous art deco New...",0.965930,"The deal was for $150 million, according to a ...",The Chrysler Building has been sold for $150 m...,0.810819,0.077465,0.112676
1,health,dr karen bell is the chair of the certificatio...,0.481145,dr karen bell is the chair of the certificati...,karen bell is the chair certification commissi...,0.443118,0.241935,0.387097
2,health_today,The most sophisticated form of HIE creates sem...,0.825120,The most sophisticated form of HIE creates sem...,The Indiana Health Information Exchange is the...,0.787513,0.113924,0.116456
3,health_today_1,"In 2013, Grahame Greive, the leader of the gl...",0.930182,The HL7 task force FHIR does not attempt to ad...,There are many strongly held points of view ab...,1.000000,0.135135,0.110565
